In [1]:
import pandas as pd
import requests
import json

In [2]:
insee_dep = pd.read_csv("./preprocessing_data/in_data/DEPARTEMENT.csv")

In [5]:
import math

def calculate_zoom_level(lat1, lon1, lat2, lon2, map_width_px, map_height_px, tile_size_px=256):
    """
    Calculate the optimal zoom level for a bounding box.

    lat1, lon1: Coordinates of one corner of the bounding box
    lat2, lon2: Coordinates of the opposite corner of the bounding box
    map_width_px, map_height_px: Dimensions of the map in pixels
    tile_size_px: Size of a map tile in pixels (default is 256)

    Returns the optimal zoom level.
    """
    # Calculate the horizontal and vertical degrees per pixel
    lat_diff = abs(lat1 - lat2)
    lon_diff = abs(lon1 - lon2)

    lat_degrees_per_pixel = lat_diff / map_height_px
    lon_degrees_per_pixel = lon_diff / map_width_px

    # Calculate the horizontal and vertical degrees per tile
    lat_degrees_per_tile = lat_degrees_per_pixel * tile_size_px
    lon_degrees_per_tile = lon_degrees_per_pixel * tile_size_px

    # Determine the maximum degrees per tile for the given latitude
    max_degrees_per_tile = max(lat_degrees_per_tile, lon_degrees_per_tile)

    # Calculate the optimal zoom level based on the maximum degrees per tile
    zoom_level = math.log2(360 / max_degrees_per_tile)

    return int(zoom_level)

# Coordinates of the bounding box
lat1 = 40.0
lon1 = -75.0
lat2 = 42.0
lon2 = -73.0

# Dimensions of the map in pixels
map_width_px = 800
map_height_px = 600

# Calculate the optimal zoom level
zoom_level = calculate_zoom_level(lat1, lon1, lat2, lon2, map_width_px, map_height_px)
print(f"Optimal Zoom Level: {zoom_level}")


Optimal Zoom Level: 8


In [ ]:
dpt_name = "Ain"

url = f"https://nominatim.openstreetmap.org/?adressdetails=1&q={dpt_name.replace(' ','+')}&format=json&limit=1"

data = requests.get(url).json()[0]

In [6]:
data


{'place_id': 297800523,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 7387,
 'lat': '46.06551335',
 'lon': '5.28478031423462',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 12,
 'importance': 0.6189198530506933,
 'addresstype': 'county',
 'name': 'Ain',
 'display_name': 'Ain, Auvergne-Rhône-Alpes, France métropolitaine, France',
 'boundingbox': ['45.6107563', '46.5199006', '4.7281684', '6.1700273']}

In [7]:
dpt_osm_info = {}

max_width_px = 600
max_height_px = 600

for dpt_code,dpt in zip(insee_dep["INSEE_DEP"],insee_dep["NOM"]):

    print(dpt_code)

    url = f"https://nominatim.openstreetmap.org/?adressdetails=1&q={dpt.replace(' ','+')}&format=json&limit=1"
    data = requests.get(url).json()[0]

    lat1, lat2, lon1, lon2 = [float(x) for x in data["boundingbox"]] 

    zoom_level = calculate_zoom_level(lat1, lon1, lat2, lon2, map_width_px, map_height_px)

    dpt_osm_info[dpt_code] = {
        "lat" : data["lat"],
        "lon" : data["lon"],
        "zoom_level" : zoom_level
    }


    




01
02
2A
2B
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
971
972
976
974
973
978


In [15]:
for it in dpt_osm_info.keys():
    dpt_osm_info[it]["lat"] = float(dpt_osm_info[it]["lat"])   
    dpt_osm_info[it]["lon"] = float(dpt_osm_info[it]["lon"])

In [16]:
dpt_osm_info

{'01': {'lat': 46.06551335, 'lon': 5.28478031423462, 'zoom_level': 9},
 '02': {'lat': 49.453285449999996, 'lon': 3.606899003594057, 'zoom_level': 9},
 '2A': {'lat': 41.87340825, 'lon': 9.0087052196875, 'zoom_level': 9},
 '2B': {'lat': 42.42196975, 'lon': 9.100906549656115, 'zoom_level': 9},
 '03': {'lat': 46.36746405, 'lon': 3.163882848311948, 'zoom_level': 9},
 '04': {'lat': 44.1640832, 'lon': 6.187851538609079, 'zoom_level': 9},
 '05': {'lat': 44.6564666, 'lon': 6.352024584507948, 'zoom_level': 9},
 '06': {'lat': 43.9210587, 'lon': 7.1790785, 'zoom_level': 9},
 '07': {'lat': 44.815194000000005, 'lon': 4.3986524702343965, 'zoom_level': 9},
 '08': {'lat': 49.69801175, 'lon': 4.671600518245179, 'zoom_level': 9},
 '09': {'lat': 42.9455368, 'lon': 1.4065544156065486, 'zoom_level': 9},
 '10': {'lat': 48.3201921, 'lon': 4.1905396615047525, 'zoom_level': 9},
 '11': {'lat': 43.0542733, 'lon': 2.512471457499548, 'zoom_level': 9},
 '12': {'lat': 44.315857449999996, 'lon': 2.5065697302419823, 'z

In [17]:
with open("./carte_grand_public/data/dpt_zoom_info.json", "w") as f:
    json.dump(dpt_osm_info, f, indent=4)

### Modifications à posteriori
# Guyane > bien centrer sur Guyane française
# modifier niveau de zoom mayotte
# rajouter tom